<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Architecture" data-toc-modified-id="Architecture-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Architecture</a></span><ul class="toc-item"><li><span><a href="#Name:-&quot;super_hydro&quot;" data-toc-modified-id="Name:-&quot;super_hydro&quot;-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Name: "super_hydro"</a></span></li></ul></li><li><span><a href="#Configuration" data-toc-modified-id="Configuration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Communication" data-toc-modified-id="Communication-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Communication</a></span></li><li><span><a href="#Logging" data-toc-modified-id="Logging-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Logging</a></span></li><li><span><a href="#References" data-toc-modified-id="References-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Architecture

Here we describe our basic architecture of our application, including configuration and communication protocols.  Our application consists of a server and multiple clients.  The server performs the computation, while the clients display the results and provide real-time inputs and controls.

For stability, we use [ØMQ] via the [pyzmq](https://pyzmq.readthedocs.io/en/latest/) library to implement the base level communication layer and define a simple protocol.

[ØMQ]: http://zeromq.org

## Name: "super_hydro"

As of 9 September 2018, there are only 69 matches on Google for the search term ["super_hydro"](https://www.google.com/search?q=%22super_hydro%22) (with quotes) and most are squirreled away (email addresses, etc.)  This seems like a safe choice for a project name.  For comparison ["SuperHydro"](https://www.google.com/search?q=%22SuperHydro%22) returned about 21k matches.

* Environment variables will be prefixed with `SUPER_HYDRO_` to minimize conflicts.
* The main module is `super_hydro`.
* Config files will use `super_hydro`.

# Configuration

Configuration takes place in several layers.  From lowest to highest precedence, we have:

1. Defaults hard-coded into the program.
2. Environmental variables.
3. Configuration files.
4. Command-line options.

The [ConfigArgParse](https://github.com/bw2/ConfigArgParse) library seems to provide this behavior out of the box, so we start with this.  A couple of comments:

* Options with a long name `--option` can be set in a config file with the same name.
* Options without `-` or `--` are positional.
* Sections are not used (by design) but can be included as comments in the config files.

Configuration options are specified in the [`super_hydro/config.py`](/edit/super_hydro/config.py) file which reads these from various config files (`~/.config/super_hydro.conf` or `./super_hydro.conf` preferred), environment variables, or command-line arguments.

*An alternative [strategy suggested by Von Welch](http://blog.vwelch.com/2011/04/combining-configparser-and-argparse.html) combines [`configparser`](https://docs.python.org/3/library/configparser.html) for reading the config files with [`argparse`](https://docs.python.org/3/library/argparse.html) for processing the command line.*

To configure the communication layer, the server must have an IP address (URL) specifying where it is located, and a port on which it listens for connections.  These are specified in the application config file.

# Communication

To simplify communication we use [ØMQ] via the [pyzmq](https://pyzmq.readthedocs.io/en/latest/) library to implement the base level communication layer and define a simple protocol through `Client` and `Server` classes in [`super_hydro/communication.py`](/edit/super_hydro/communication.py).  Clients can issue the following commands to the server:

* `obj = get(msg)`: This requests data from the server and will return the object sent back by the server.
* `request(msg)`: This asks the server to do something but does not request data from the server.
* `send(msg, obj)`: This will send data to the server.

Note: all messages should be byte objects, not strings (i.e. `msg=b"Window.size"` not `msg="Window.size"`).

Data (`obj`s) are serialized using `json.dumps(obj).encode()` and deserialized by `json.loads(msg.decode())`.

[ØMQ]: http://zeromq.org

Currently the code employs the `zmq.REQ` (Server) and `zmq.REP` (Client) socket types for which each message requires a send and receive.  This is simply [described here](https://learning-0mq-with-pyzmq.readthedocs.io/en/latest/pyzmq/patterns/client_server.html) but may not allow multiple clients to connect to the server.  (It definitely allows a client to connect with multiple servers, but this is not the pattern we want.)

In [26]:
import numpy as np
A = np.random.random((128, 128)) + 1j
%timeit A.tostring()
%timeit A.tobytes()

21.4 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
21.5 µs ± 551 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
s = A.tobytes()
B = np.frombuffer(A, dtype=complex).reshape(A.shape)
C = np.frombuffer(s, dtype=complex).reshape(A.shape)
assert np.allclose(A, C)
assert np.allclose(A, B)

%timeit np.frombuffer(A, dtype=complex).reshape(A.shape)
%timeit np.frombuffer(s, dtype=complex).reshape(A.shape)



18.8 µs ± 323 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.59 µs ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.18 µs ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Logging

Logging is currently done through the [`logging`](https://docs.python.org/3/library/logging.html) module through the `Logger` class in [`super_hydro/utils.py`](/edit/super_hydro/utils.py).  Two mechanisms are provided in the code:

* Output a message at a specified logging level (default is INFO):

  ```python
  log(msg, level=logging.INFO)
  ```
  
* Frame a task with messages at the start and end with a specified logging level:

  ```python
  with log_task(msg, level=logging.INFO):
      ...
  ```

For the clients, we might want to use the logging facilities of the front end.

# References

* [Which is the best way to allow configuration options be overridden at the command line in Python?
](https://stackoverflow.com/questions/3609852/which-is-the-best-way-to-allow-configuration-options-be-overridden-at-the-comman/5826167)
* https://github.com/lebedov/msgpack-numpy